# Interactive Equity Valuation Notebook

**Objective:** This notebook provides a basic, interactive framework for exploring common equity valuation techniques, including Discounted Cash Flow (DCF), Trading Comparables, and basic Bond Pricing concepts. It uses a sample dataset of mega-cap public equity names.

**Disclaimer:** This notebook is for educational and illustrative purposes only. The models are simplified and use hypothetical or publicly available data that may not be current or perfectly accurate. It is NOT financial advice.

## 1. Setup and Data Loading

First, we'll import necessary Python libraries and load our sample dataset of mega-cap companies.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
from ipywidgets import interact, widgets, fixed

# Set a default theme for plotly charts
pio.templates.default = "plotly_white"

# Load the dataset
try:
    companies_df = pd.read_csv('data/mega_cap_equity_data.csv')
except FileNotFoundError:
    print("Error: 'data/mega_cap_equity_data.csv' not found. Make sure the CSV file is in the correct directory.")
    # Create an empty DataFrame with expected columns if file not found, so notebook can still partially run
    companies_df = pd.DataFrame(columns=['Ticker', 'CompanyName', 'Industry', 'RevenueUSD_M', 'EBITDA_USD_M', 'NetDebtUSD_M', 'MarketCapUSD_M', 'StockPriceUSD', 'SharesOutstanding_M'])

def display_company_data(df):
    if not df.empty:
        print("Sample Company Data Loaded:")
        print(df.head())
    else:
        print("Company data is empty or could not be loaded.")

display_company_data(companies_df)

sns.set_style("whitegrid")

### 1.1. Illustrative Web Scraping Code (Conceptual)

The cell below contains conceptual code illustrating how one *might* attempt to scrape financial data using `yfinance` (for market data) and `requests`/`BeautifulSoup` (for fundamentals - though this is much more complex and site-specific in reality). 

**Note:** This scraping code is for demonstration only and may **not run correctly** in all environments (especially not in a static HTML embed or restricted online sandboxes due to web access limitations, CORS, and website structure changes). **Our notebook will primarily use the pre-loaded CSV data for reliability.**

In [ ]:
# --- CONCEPTUAL SCRAPING CODE - FOR ILLUSTRATION ONLY --- 
# import yfinance as yf
# import requests
# from bs4 import BeautifulSoup

# def get_conceptual_market_data(ticker_symbol):
#     try:
#         stock = yf.Ticker(ticker_symbol)
#         info = stock.info
#         market_cap = info.get('marketCap', 0)
#         current_price = info.get('currentPrice', info.get('previousClose', 0))
#         shares_outstanding = info.get('sharesOutstanding', 0)
#         print(f"Data for {ticker_symbol}: MCAP={market_cap}, Price={current_price}, Shares={shares_outstanding}")
#         return market_cap, current_price, shares_outstanding
#     except Exception as e:
#         print(f"Could not fetch yfinance data for {ticker_symbol}: {e}")
#         return 0, 0, 0

# def get_conceptual_fundamental_data(ticker_symbol):
#     # This is highly simplified and likely to break with website changes.
#     # Real fundamental scraping is complex (e.g., using SEC Edgar APIs or paid services).
#     headers = {'User-Agent': 'Mozilla/5.0'}
#     url_financials = f"https://finance.yahoo.com/quote/{ticker_symbol}/financials?p={ticker_symbol}"
#     try:
#         response = requests.get(url_financials, headers=headers)
#         soup = BeautifulSoup(response.text, 'html.parser')
#         # Example: Try to find Total Revenue (this will be very fragile)
#         # revenue_label_element = soup.find('div', title='Total Revenue')
#         # if revenue_label_element and revenue_label_element.parent and revenue_label_element.parent.find_next_sibling():
#         #     revenue_value_element = revenue_label_element.parent.find_next_sibling().find('span')
#         #     if revenue_value_element:
#         #         revenue_str = revenue_value_element.text.replace(',', '')
#         #         print(f"Found conceptual revenue for {ticker_symbol}: {revenue_str}")
#         #         return int(revenue_str) * 1000 # Assuming values are in thousands
#         print(f"Conceptual fundamental scraping for {ticker_symbol} is illustrative and not fully implemented.")
#     except Exception as e:
#         print(f"Could not fetch fundamental data for {ticker_symbol}: {e}")
#     return 0

# # Example usage (conceptual):
# # get_conceptual_market_data('AAPL')
# # get_conceptual_fundamental_data('AAPL')
print("Conceptual web scraping code is for illustration. Active scraping is not performed in this notebook.")

## 2. Interactive Discounted Cash Flow (DCF) Model

A simplified DCF model. We'll project Free Cash Flow to Firm (FCFF) for a few years, calculate a terminal value, and discount these back to the present to find an implied Enterprise Value (EV) and Equity Value.

In [ ]:
def calculate_dcf(company_name, current_ebitda, revenue_growth_rate, ebitda_margin, tax_rate, depr_amort_percent_revenue, capex_percent_revenue, wc_percent_revenue, discount_rate_wacc, terminal_growth_rate, years_to_project=5):
    if companies_df[companies_df['CompanyName'] == company_name].empty and current_ebitda == 0:
        print(f"Company {company_name} not found in dataset or no EBITDA provided. Please select a valid company or provide current EBITDA.")
        return
    
    # If a company is selected, use its data, otherwise use manual input
    if not companies_df[companies_df['CompanyName'] == company_name].empty:
        company_data = companies_df[companies_df['CompanyName'] == company_name].iloc[0]
        last_revenue = company_data['RevenueUSD_M']
        current_ebitda_val = current_ebitda if current_ebitda != 0 else company_data['EBITDA_USD_M'] 
        net_debt = company_data['NetDebtUSD_M']
        shares_outstanding = company_data['SharesOutstanding_M']
    else: # Manual input case
        last_revenue = current_ebitda / (ebitda_margin / 100.0) if ebitda_margin !=0 else current_ebitda * 5
        current_ebitda_val = current_ebitda
        net_debt = 0
        shares_outstanding = 1000
        print(f"Using manually entered EBITDA: ${current_ebitda_val:,.2f}M and estimated Revenue: ${last_revenue:,.2f}M")

    fcffs = []
    revenues = [last_revenue]
    ebitdas = [current_ebitda_val]

    for year in range(1, years_to_project + 1):
        revenue = revenues[-1] * (1 + revenue_growth_rate / 100.0)
        ebitda = revenue * (ebitda_margin / 100.0)
        
        depreciation_amortization = revenue * (depr_amort_percent_revenue / 100.0)
        ebit = ebitda - depreciation_amortization
        nopat = ebit * (1 - tax_rate / 100.0)
        
        capex = revenue * (capex_percent_revenue / 100.0)
        delta_wc = (revenue - revenues[-1]) * (wc_percent_revenue / 100.0)
        
        fcff = nopat + depreciation_amortization - capex - delta_wc
        fcffs.append(fcff)
        revenues.append(revenue)
        ebitdas.append(ebitda)

    # Terminal Value (Gordon Growth Model)
    terminal_fcff = fcffs[-1] * (1 + terminal_growth_rate / 100.0)
    terminal_value = terminal_fcff / (discount_rate_wacc / 100.0 - terminal_growth_rate / 100.0)
    
    # Discounting
    pv_fcffs = [fcff / ((1 + discount_rate_wacc / 100.0)**(year + 1)) for year, fcff in enumerate(fcffs)]
    pv_terminal_value = terminal_value / ((1 + discount_rate_wacc / 100.0)**years_to_project)
    
    enterprise_value = sum(pv_fcffs) + pv_terminal_value
    equity_value = enterprise_value - net_debt
    implied_share_price = equity_value / shares_outstanding if shares_outstanding else 0
    
    # --- Output --- 
    print(f"\n--- DCF Results for {company_name if company_name else 'Manual Input'} ---")
    print(f"Implied Share Price: ${implied_share_price:.2f}")
    print(f"Enterprise Value (EV): ${enterprise_value:,.2f}M")
    print(f"Implied Equity Value: ${equity_value:,.2f}M")
    print(f"Terminal Value (PV): ${pv_terminal_value:,.2f}M")
    print(f"Projected FCFFs (PV): {[round(f, 2) for f in pv_fcffs]}")
    
    # --- Plotting with Plotly --- 
    plot_df = pd.DataFrame({'Year': range(1, years_to_project + 1), 'FCFF': fcffs})
    fig = px.bar(plot_df, x='Year', y='FCFF', title=f'Projected FCFF for {company_name if company_name else "Manual Input"}',
                   labels={'FCFF': 'Projected FCFF (USD Millions)', 'Year': 'Projection Year'},
                   text_auto='.2s', hover_data={'FCFF': ':.2f'})
    fig.update_traces(textangle=0, textposition='outside')
    fig.show()

# --- Widgets for Interaction --- 
company_names_list = [''] + sorted(companies_df['CompanyName'].unique().tolist()) # Add blank option for manual input

interact(
    calculate_dcf,
    company_name=widgets.Dropdown(options=company_names_list, description='Company:'),
    current_ebitda=widgets.FloatText(value=10000, description='Current EBITDA (M):', continuous_update=False),
    revenue_growth_rate=widgets.FloatSlider(min=0, max=20, step=0.5, value=5, description='Revenue Growth (%):'),
    ebitda_margin=widgets.FloatSlider(min=5, max=50, step=1, value=25, description='EBITDA Margin (%):'),
    tax_rate=widgets.FloatSlider(min=15, max=35, step=1, value=21, description='Tax Rate (%):'),
    depr_amort_percent_revenue=widgets.FloatSlider(min=1, max=15, step=0.5, value=4.0, description='D&A (% of Rev):'),
    capex_percent_revenue=widgets.FloatSlider(min=1, max=20, step=0.5, value=5, description='CapEx (% of Rev):'),
    wc_percent_revenue=widgets.FloatSlider(min=0, max=15, step=0.5, value=3, description='Chg WC (% of Chg Rev):'),
    discount_rate_wacc=widgets.FloatSlider(min=5, max=15, step=0.25, value=8.0, description='WACC (%):'),
    terminal_growth_rate=widgets.FloatSlider(min=0, max=5, step=0.1, value=2.0, description='Terminal Growth (%):'),
    years_to_project=fixed(5)
);

## 3. Advanced Trading Comparables Analysis

Here, we analyze how a company is valued relative to its peers using an interactive scatter plot. This allows us to visualize multiple metrics at once: EV/EBITDA on the x-axis, P/E on the y-axis, industry by color, and market cap by bubble size.

### 3.1. A Note on P/E Calculation
Since our dataset does not contain Net Income, we will estimate it to calculate a more realistic P/E ratio. We'll use the following formula as a rough proxy:

`Estimated Net Income = EBITDA * (1 - Typical D&A Rate) * (1 - Typical Tax Rate)`

This is more robust than simply taking a fraction of EBITDA and provides a better basis for comparison. We will assume a typical D&A rate of 15% of EBITDA and a tax rate of 21% for this calculation.

In [ ]:
def plot_interactive_comps(industry_filter='All'):
    if companies_df.empty:
        print("No company data available for comparables analysis.")
        return

    # --- Enhanced Multiple Calculations ---
    temp_df = companies_df.copy()
    temp_df['EV_USD_M'] = temp_df['MarketCapUSD_M'] + temp_df['NetDebtUSD_M']
    temp_df['EV_EBITDA'] = temp_df.apply(lambda x: x['EV_USD_M'] / x['EBITDA_USD_M'] if x['EBITDA_USD_M'] > 0 else np.nan, axis=1)
    
    # Estimate Net Income for a more realistic P/E
    assumed_dna_rate_of_ebitda = 0.15
    assumed_tax_rate = 0.21
    temp_df['Estimated_Net_Income'] = temp_df['EBITDA_USD_M'] * (1 - assumed_dna_rate_of_ebitda) * (1 - assumed_tax_rate)
    temp_df['P_E'] = temp_df.apply(lambda x: x['MarketCapUSD_M'] / x['Estimated_Net_Income'] if x['Estimated_Net_Income'] > 0 else np.nan, axis=1)

    # Filter out extreme outliers for better visualization
    temp_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    temp_df.dropna(subset=['EV_EBITDA', 'P_E'], inplace=True)
    pe_cap = temp_df['P_E'].quantile(0.95) # Cap P/E at the 95th percentile
    ev_ebitda_cap = temp_df['EV_EBITDA'].quantile(0.95)
    temp_df = temp_df[(temp_df['P_E'] < pe_cap) & (temp_df['EV_EBITDA'] < ev_ebitda_cap)]
    
    plot_df = temp_df if industry_filter == 'All' else temp_df[temp_df['Industry'] == industry_filter]
    
    if plot_df.empty:
        print(f"No companies with valid, non-outlier data found for industry: {industry_filter}")
        return

    # --- Interactive Plotting with Plotly ---
    fig = px.scatter(
        plot_df,
        x='EV_EBITDA',
        y='P_E',
        size='MarketCapUSD_M',
        color='Industry',
        hover_name='CompanyName',
        hover_data={'Ticker': True, 'MarketCapUSD_M': ':.2s', 'EV_EBITDA': ':.2f', 'P_E': ':.2f'},
        title=f'Trading Comparables Analysis ({industry_filter})',
        labels={'EV_EBITDA': 'EV / EBITDA Multiple', 'P_E': 'P / E Multiple (Estimated)'}
    )
    
    fig.update_layout(legend_title_text='Industry', height=600)
    fig.show()

industry_list = ['All'] + sorted(companies_df['Industry'].unique().tolist())
interact(plot_interactive_comps, industry_filter=widgets.Dropdown(options=industry_list, description='Industry:'));

## 4. Basic Bond Pricing

Illustrating the relationship between bond price, coupon rate, yield to maturity (YTM), and time to maturity.

In [ ]:
def calculate_bond_price(face_value, coupon_rate_percent, ytm_percent, years_to_maturity, payments_per_year=2):
    coupon_rate = coupon_rate_percent / 100.0
    ytm = ytm_percent / 100.0
    
    coupon_payment = (face_value * coupon_rate) / payments_per_year
    total_payments = years_to_maturity * payments_per_year
    discount_rate_periodic = ytm / payments_per_year
    
    pv_coupons = 0
    for t in range(1, int(total_payments) + 1):
        pv_coupons += coupon_payment / ((1 + discount_rate_periodic)**t)
        
    pv_face_value = face_value / ((1 + discount_rate_periodic)**total_payments)
    
    bond_price = pv_coupons + pv_face_value
    
    print(f"--- Bond Pricing Result ---")
    print(f"Face Value: ${face_value:,.2f}")
    print(f"Annual Coupon Rate: {coupon_rate_percent:.2f}%")
    print(f"Yield to Maturity (YTM): {ytm_percent:.2f}%")
    print(f"Years to Maturity: {years_to_maturity}")
    print(f"Periodic Coupon Payment: ${coupon_payment:.2f}")
    print(f"Calculated Bond Price: ${bond_price:,.2f}")
    
    if bond_price > face_value:
        print("Bond is trading at a PREMIUM (Coupon Rate > YTM)")
    elif bond_price < face_value:
        print("Bond is trading at a DISCOUNT (Coupon Rate < YTM)")
    else:
        print("Bond is trading at PAR (Coupon Rate = YTM)")

interact(
    calculate_bond_price,
    face_value=widgets.FloatText(value=1000, description='Face Value:'),
    coupon_rate_percent=widgets.FloatSlider(min=0, max=15, step=0.25, value=5.0, description='Coupon Rate (%):'),
    ytm_percent=widgets.FloatSlider(min=0, max=15, step=0.25, value=6.0, description='YTM (%):'),
    years_to_maturity=widgets.IntSlider(min=1, max=30, step=1, value=10, description='Years to Maturity:'),
    payments_per_year=fixed(2)
);

## 5. Conclusion

This notebook provided a brief, interactive introduction to some core equity valuation and bond pricing concepts. 

**Further Learning from the Repository:**
*   For detailed financial modeling best practices, see `Financial_Modeling/Best_Practices/`.
*   For in-depth DCF theory, refer to `Financial_Modeling/Valuation_Applications/DCF_Modeling_Overview.md`.
*   For market context, explore `Market_Analysis_Quick_Start/`.

Remember to download this `.ipynb` file to run and modify the code interactively in your own Jupyter environment if you are viewing this as static HTML.